In [0]:
%pip install great-expectations==0.18.12
dbutils.library.restartPython()

In [0]:
from pyspark.sql.functions import col, when, min, max
import great_expectations as ge
from great_expectations.dataset import SparkDFDataset
from datetime import datetime

print(f"Great Expectations version: {ge.__version__}")


In [0]:
from pyspark.sql.functions import col, min, max, when, count
from datetime import datetime

# Validación para revenue_by_day
print("Validating: gold.revenue_by_day")
df_revenue = spark.table("gold.revenue_by_day")

total_rows = df_revenue.count()
print(f"Total rows in table: {total_rows}")

null_checks = {}
for col_name in ["purchase_date","total_revenue","total_orders","total_unique_orders"]:
    null_count = df_revenue.filter(col(col_name).isNull()).count()
    null_checks[col_name] = null_count
    status = "passed" if null_count == 0 else "failed"
    print(f"Null check for column {col_name}: {status} - {null_count} nulls found")

negative_revenue = df_revenue.filter(col("total_revenue") < 0).count()
status = "passed" if negative_revenue == 0 else "failed"
print(f"Revenue validation (>= 0): {status} - {negative_revenue} invalid records")

invalid_logic = df_revenue.filter(col("total_orders") < col("total_unique_orders")).count()
status = "passed" if invalid_logic == 0 else "failed"
print(f"Business logic (orders >= customers): {status} - {invalid_logic} invalid records")

date_stats = df_revenue.select(
    min("purchase_date").alias("min_date"),
    max("purchase_date").alias("max_date")
).collect()[0]
print(f"Date range validation: passed - {date_stats['min_date']} to {date_stats['max_date']}")

checks_passed = sum([v == 0 for v in null_checks.values()]) + int(negative_revenue == 0) + int(invalid_logic == 0)
total_checks = len(null_checks) + 2
success_rate = (checks_passed / total_checks) * 100
print(f"\nSummary: {checks_passed}/{total_checks} checks passed ({success_rate:.0f}%)")

revenue_checks = (checks_passed, total_checks)

# Validación para customer_rfm_segments
print("\n" + "="*60)
print("Validating: gold.customer_rfm_segments")

df_rfm = spark.table("gold.customer_rfm_segments")

total_rows = df_rfm.count()
print(f"Total rows in table: {total_rows}")

# Primero verificar qué columnas existen realmente
available_cols = df_rfm.columns
print(f"Available columns: {available_cols}")

# Solo validar columnas que existen
rfm_columns_to_check = ["customer_id", "segment", "total_spent", "total_orders", 
                       "recency_score", "frequency_score", "monetary_score", "days_since_last_order"]
existing_cols = [col for col in rfm_columns_to_check if col in df_rfm.columns]

print(f"Validating columns: {existing_cols}")

null_checks = {}
for col_name in existing_cols:
    null_count = df_rfm.filter(col(col_name).isNull()).count()
    null_checks[col_name] = null_count
    status = "passed" if null_count == 0 else "failed"
    print(f"Null check for column {col_name}: {status} - {null_count} nulls found")

# CHECK 2: Customer_id uniqueness (solo si existe)
if "customer_id" in df_rfm.columns:
    duplicate_customers = df_rfm.groupBy("customer_id").count().filter(col("count") > 1).count()
    status = "passed" if duplicate_customers == 0 else "failed"
    print(f"Uniqueness check (customer_id): {status} - {duplicate_customers} duplicates found")
else:
    duplicate_customers = 0
    print("Uniqueness check (customer_id): skipped - column not found")

# CHECK 3: RFM scores in range [1-5] (solo si existen)
score_checks = {}
for score_col in ["recency_score", "frequency_score", "monetary_score"]:
    if score_col in df_rfm.columns:
        invalid_scores = df_rfm.filter((col(score_col) < 1) | (col(score_col) > 5)).count()
        score_checks[score_col] = invalid_scores
        status = "passed" if invalid_scores == 0 else "failed"
        print(f"Range validation {score_col} [1-5]: {status} - {invalid_scores} invalid records")
    else:
        score_checks[score_col] = 0
        print(f"Range validation {score_col} [1-5]: skipped - column not found")

# CHECK 4: Recency >= 0 (solo si existe)
if "days_since_last_order" in df_rfm.columns:
    negative_recency = df_rfm.filter(col("days_since_last_order") < 0).count()
    status = "passed" if negative_recency == 0 else "failed"
    print(f"Recency validation (>= 0): {status} - {negative_recency} invalid records")
else:
    negative_recency = 0
    print("Recency validation (>= 0): skipped - column not found")

# CHECK 5: Frequency >= 1 (solo si existe)
if "total_orders" in df_rfm.columns:
    zero_frequency = df_rfm.filter(col("total_orders") < 1).count()
    status = "passed" if zero_frequency == 0 else "failed"
    print(f"Frequency validation (>= 1): {status} - {zero_frequency} invalid records")
else:
    zero_frequency = 0
    print("Frequency validation (>= 1): skipped - column not found")

# CHECK 6: Monetary > 0 (solo si existe)
if "total_spent" in df_rfm.columns:
    zero_monetary = df_rfm.filter(col("total_spent") <= 0).count()
    status = "passed" if zero_monetary == 0 else "failed"
    print(f"Monetary validation (> 0): {status} - {zero_monetary} invalid records")
else:
    zero_monetary = 0
    print("Monetary validation (> 0): skipped - column not found")

# CHECK 7: Valid segments (solo si existe)
if "segment" in df_rfm.columns:
    valid_segments = [
        "Champions", "Loyal Customers", "Potential Loyalist",
        "New Customers", "Promising", "Need Attention",
        "About to Sleep", "At Risk", "Can't Lose Them",
        "Hibernating", "Lost"
    ]
    invalid_segments = df_rfm.filter(~col("segment").isin(valid_segments)).count()
    status = "passed" if invalid_segments == 0 else "failed"
    print(f"Segment validation (valid values): {status} - {invalid_segments} invalid records")
else:
    invalid_segments = 0
    print("Segment validation (valid values): skipped - column not found")

# Summary
rfm_checks_passed = (
    sum([v == 0 for v in null_checks.values()]) +
    (duplicate_customers == 0) +
    sum([v == 0 for v in score_checks.values()]) +
    (negative_recency == 0) +
    (zero_frequency == 0) +
    (zero_monetary == 0) +
    (invalid_segments == 0)
)

rfm_total_checks = (
    len(null_checks) + 
    1 +  # uniqueness check
    len(score_checks) + 
    4    # recency, frequency, monetary, segment checks
)

rfm_success_rate = (rfm_checks_passed / rfm_total_checks) * 100
print(f"\nSummary: {rfm_checks_passed}/{rfm_total_checks} checks passed ({rfm_success_rate:.0f}%)")

rfm_checks = (rfm_checks_passed, rfm_total_checks)

# Validación para product_performance_ranking
print("\n" + "="*60)
print("Validating: gold.product_category_summary")

df_products = spark.table("gold.product_category_summary")

total_rows = df_products.count()
print(f"Total rows in table: {total_rows}")

# Verificar columnas disponibles
available_cols = df_products.columns
print(f"Available columns: {available_cols}")

# Columnas específicas para productos
product_columns_to_check = ["product_id", "item_revenue", "total_orders", "revenue_rank", "avg_order_value"]
existing_prod_cols = [col for col in product_columns_to_check if col in df_products.columns]

print(f"Validating columns: {existing_prod_cols}")

null_checks = {}
for col_name in existing_prod_cols:
    null_count = df_products.filter(col(col_name).isNull()).count()
    null_checks[col_name] = null_count
    status = "passed" if null_count == 0 else "failed"
    print(f"Null check for column {col_name}: {status} - {null_count} nulls found")

# CHECK 2: Product_id uniqueness
if "product_id" in df_products.columns:
    duplicate_products = df_products.groupBy("product_id").count().filter(col("count") > 1).count()
    status = "passed" if duplicate_products == 0 else "failed"
    print(f"Uniqueness check (product_id): {status} - {duplicate_products} duplicates found")
else:
    duplicate_products = 0
    print("Uniqueness check (product_id): skipped - column not found")

# CHECK 3: Revenue > 0
if "item_revenue" in df_products.columns:
    zero_revenue = df_products.filter(col("item_revenue") <= 0).count()
    status = "passed" if zero_revenue == 0 else "failed"
    print(f"Revenue validation (> 0): {status} - {zero_revenue} invalid records")
else:
    zero_revenue = 0
    print("Revenue validation (> 0): skipped - column not found")

# CHECK 4: Orders >= 1
if "total_orders" in df_products.columns:
    zero_orders = df_products.filter(col("total_orders") < 1).count()
    status = "passed" if zero_orders == 0 else "failed"
    print(f"Orders validation (>= 1): {status} - {zero_orders} invalid records")
else:
    zero_orders = 0
    print("Orders validation (>= 1): skipped - column not found")

# CHECK 5: Ranking >= 1
if "revenue_rank" in df_products.columns:
    invalid_rank = df_products.filter(col("revenue_rank") < 1).count()
    status = "passed" if invalid_rank == 0 else "failed"
    print(f"Ranking validation (>= 1): {status} - {invalid_rank} invalid records")
else:
    invalid_rank = 0
    print("Ranking validation (>= 1): skipped - column not found")

# CHECK 6: AOV consistency
if "avg_order_value" in df_products.columns and "item_revenue" in df_products.columns and "total_orders" in df_products.columns:
    df_aov_check = df_products.withColumn(
        "aov_calculated", 
        col("item_revenue") / col("total_orders")
    ).withColumn(
        "aov_diff",
        when(col("avg_order_value").isNotNull(), 
             col("avg_order_value") - col("aov_calculated")
        ).otherwise(0)
    )
    aov_inconsistent = df_aov_check.filter(
        (col("aov_diff") > 0.01) | (col("aov_diff") < -0.01)
    ).count()
    status = "passed" if aov_inconsistent == 0 else "failed"
    print(f"AOV consistency validation: {status} - {aov_inconsistent} inconsistent records")
else:
    aov_inconsistent = 0
    print("AOV consistency validation: skipped - required columns not found")

# Summary
prod_checks_passed = (
    sum([v == 0 for v in null_checks.values()]) +
    (duplicate_products == 0) +
    (zero_revenue == 0) +
    (zero_orders == 0) +
    (invalid_rank == 0) +
    (aov_inconsistent == 0)
)

prod_total_checks = len(null_checks) + 5
prod_success_rate = (prod_checks_passed / prod_total_checks) * 100
print(f"\nSummary: {prod_checks_passed}/{prod_total_checks} checks passed ({prod_success_rate:.0f}%)")

prod_checks = (prod_checks_passed, prod_total_checks)

# Resumen final
print("\n" + "="*60)
print("FINAL VALIDATION SUMMARY")
print("="*60)

total_checks_all = revenue_checks[1] + rfm_checks[1] + prod_checks[1]
total_passed_all = revenue_checks[0] + rfm_checks[0] + prod_checks[0]
overall_success = (total_passed_all / total_checks_all) * 100

print(f"""
Total validations executed: {total_checks_all}
Validations passed: {total_passed_all}
Validations failed: {total_checks_all - total_passed_all}
Overall success rate: {overall_success:.1f}%

Breakdown by table:
  - revenue_by_day: {success_rate:.0f}% ({revenue_checks[0]}/{revenue_checks[1]})
  - customer_rfm_segments: {rfm_success_rate:.0f}% ({rfm_checks[0]}/{rfm_checks[1]})
  - product_performance_ranking: {prod_success_rate:.0f}% ({prod_checks[0]}/{prod_checks[1]})

Validation timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
""")

if overall_success >= 95:
    print("Status: ✅ EXCELLENT - High data quality")
elif overall_success >= 80:
    print("Status: ✅ GOOD - Acceptable data quality")
else:
    print("Status: ⚠️ NEEDS ATTENTION - Review failures")

print("="*60)